# Transform data by using Spark

Apache Spark provides a distributed data processing platform that you can use to perform complex data transformations at scale.


## Load source data

Let's start by loading some historical sales order data into a dataframe.

Review the code in the cell below, which loads the sales order from all of the csv files within the **data** directory. Then click the **&#9655;** button to the left of the cell to run it.

> **Note**: The first time you run a cell in a notebook, the Spark pool must be started; which can take several minutes.

In [14]:

from spark_helper import get_spark

spark = get_spark()

# Now use spark as usual
print(spark)

In [15]:

vocabulary = spark.read.csv('data/small.csv', header=True, inferSchema=True)

print(vocabulary.show())

display(vocabulary.limit(5))

+------+--------------------+
|  Word|             Meaning|
+------+--------------------+
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
+------+--------------------+

None


DataFrame[Word: string, Meaning: string]

In [16]:
vocabulary.show(5)

+------+--------------------+
|  Word|             Meaning|
+------+--------------------+
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
+------+--------------------+
only showing top 5 rows


In [17]:
display(vocabulary.show(5))

+------+--------------------+
|  Word|             Meaning|
+------+--------------------+
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
|abacus|A counting tool u...|
+------+--------------------+
only showing top 5 rows


None

In [18]:
type(vocabulary)

pyspark.sql.classic.dataframe.DataFrame

In [19]:
# read data ffrom a csv file
customers = spark.read.csv('data/customers.csv', header=True, inferSchema=True)
customers.limit(5).show()

+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|          5153435776|    686-620-1820x944|     vmata@colon.com|     

In [20]:
from pyspark.sql.functions import concat_ws

# create a new column 'Customer Name' by concatenating 'First Name' and 'Last Name'
customers = customers.withColumn('CustomerName', concat_ws(' ', customers['First Name'], customers['Last Name']))
# Display the first 5 rows of the updated DataFrame
customers.limit(5).show()



+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+
|Index|    Customer Id|First Name|Last Name|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|  CustomerName|
+-----+---------------+----------+---------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+
|    1|DD37Cf93aecA6Dc|    Sheryl|   Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...| Sheryl Baxter|
|    2|1Ef7b82A4CAAD10|   Preston|   Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|        

In [21]:

# drop column 'Customer Name'- If it has
# customers = customers.drop('Customer Name')

# rename First Name to FirstName and Last Name to LastName
customers_renamed = customers.withColumnRenamed('First Name', 'FirstName').withColumnRenamed('Last Name', 'LastName')

# Display the first 5 rows of the updated DataFrame
customers_renamed.limit(5).show()

+-----+---------------+---------+--------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+
|Index|    Customer Id|FirstName|LastName|             Company|             City|             Country|             Phone 1|             Phone 2|               Email|Subscription Date|             Website|  CustomerName|
+-----+---------------+---------+--------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-----------------+--------------------+--------------+
|    1|DD37Cf93aecA6Dc|   Sheryl|  Baxter|     Rasmussen Group|     East Leonard|               Chile|        229.077.5154|    397.884.0519x718|zunigavanessa@smi...|       2020-08-24|http://www.stephe...| Sheryl Baxter|
|    2|1Ef7b82A4CAAD10|  Preston|  Lozano|         Vega-Gentry|East Jimmychester|            Djibouti|          51534357

In [23]:
customers_renamed.write.parquet('data/customers_renamed.parquet', mode='overwrite')


Py4JJavaError: An error occurred while calling o133.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdir

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **transformed_data** has been created, containing a file named **orders.parquet**. Then return to this notebook.

## Partition data

A common way to optimize performance when dealing with large volumes of data is to partition the data files based on one or more field values. This can significant improve performance and make it easier to filter data.

Use the following cell to derive new **Year** and **Month** fields and then save the resulting data in Parquet format, partitioned by year and month.

In [ ]:
from pyspark.sql.functions import year, month, col

dated_df = transformed_df.withColumn("Year", year(col("OrderDate"))).withColumn("Month", month(col("OrderDate")))
display(dated_df.limit(5))
dated_df.write.partitionBy("Year","Month").mode("overwrite").parquet("/partitioned_data")
print ("Transformed data saved!")

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **partitioned_data** has been created, containing a hierachy of folders in the format **Year=*NNNN*** / **Month=*N***, each containing a .parquet file for the orders placed in the corresponding year and month. Then return to this notebook.

You can read this data into a dataframe from any folder in the hierarchy, using explicit values or wildcards for partitioning fields. For example, use the following code to get the sales orders placed in 2020 for all months.

In [ ]:
orders_2020 = spark.read.parquet('/partitioned_data/Year=2020/Month=*')
display(orders_2020.limit(5))

Note that the partitioning columns specified in the file path are omitted in the resulting dataframe.

## Use SQL to transform data

Spark is a very flexible platform, and the **SQL** library that provides the dataframe also enables you to work with data using SQL semantics. You can query and transform data in dataframes by using SQL queries, and persist the results as tables - which are metadata abstractions over files.

First, use the following code to save the original sales orders data (loaded from CSV files) as a table. Technically, this is an *external* table because the **path** parameter is used to specify where the data files for the table are stored (an *internal* table is stored in the system storage for the Spark metastore and managed automatically).

In [ ]:
order_details.write.saveAsTable('sales_orders', format='parquet', mode='overwrite', path='/sales_orders_table')

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **sales_orders_table** has been created, containing parquet files for the table data. Then return to this notebook.

Now that the table has been created, you can use SQL to transform it. For example, the following code derives new Year and Month columns and then saves the results as a partitioned external table.

In [ ]:
sql_transform = spark.sql("SELECT *, YEAR(OrderDate) AS Year, MONTH(OrderDate) AS Month FROM sales_orders")
display(sql_transform.limit(5))
sql_transform.write.partitionBy("Year","Month").saveAsTable('transformed_orders', format='parquet', mode='overwrite', path='/transformed_orders_table')

In the **files** tab (which should still be open above), navigate to the root **files** container and verify that a new folder named **transformed_orders_table** has been created, containing a hierachy of folders in the format **Year=*NNNN*** / **Month=*N***, each containing a .parquet file for the orders placed in the corresponding year and month. Then return to this notebook.

Essentially you've performed the same data transformation into partitioned parquet files as s before, but by using SQL instead of native dataframe methods.

You can read this data into a dataframe from any folder in the hierarchy as before, but because the data files are also abstracted by a table in the metastore, you can query the data directly using SQL.

In [ ]:
%%sql

SELECT * FROM transformed_orders
WHERE Year = 2021
    AND Month = 1

Because these are *external* tables, you can drop the tables from the metastore without deleting the files - so the transfomed data remains available for other downstream data analytics or ingestion processes.

In [ ]:
%%sql

DROP TABLE transformed_orders;
DROP TABLE sales_orders;